In [125]:
import geopandas as gpd
import pandas as pd
from shapely.ops import substring

In [83]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
df_l = gpd.read_file(f'zip://gis/SIRGAS_SHP_logradouronbl.zip!SIRGAS_SHP_logradouronbl/SIRGAS_SHP_logradouronbl.shp')
df_class = gpd.read_file(f'zip://gis/SIRGAS_SHP_classeviariacet.zip!SIRGAS_SHP_classeviariacet/SIRGAS_SHP_classeviariacet.shp')
df_dist = gpd.read_file(f'zip://gis/SIRGAS_SHP_distrito.zip!SIRGAS_SHP_distrito/SIRGAS_SHP_distrito_polygon.shp')


In [3]:
df_l.geometry = df_l.geometry.apply(lambda x: substring(x, start_dist=0.10, end_dist=0.90, normalized=True))

In [71]:
df_l.loc[:, 'comprimento'] = df_l.geometry.length

In [4]:
df_class.geometry = df_class.geometry.apply(lambda x: substring(x, start_dist=0.10, end_dist=0.90, normalized=True))

In [18]:
df_l.explore()

In [121]:
for row, dist in df_dist.iterrows():
    print(dist.ds_nome)

    pvias = gpd.read_file(f'resultado/pre-processamentos/{dist.ds_codigo}_pre_poligono_de_vias_{dist.ds_nome.lower().replace(" ", "_")}.gpkg')
    l = df_l.clip(dist.geometry.buffer(20))
    class_viaria = df_class.clip(dist.geometry.buffer(20))

    pvias_l = pvias.sjoin(l, op='covers', how='left')[['lg_codlog', 'lg_seg_id', 'comprimento']].reset_index().groupby('index').agg(
        {
            'index': 'count',
            'lg_codlog': 'first',
            'lg_seg_id': 'first',
            'comprimento': 'sum'
        }
    ).rename(columns = {'index':'contagem'})

    pvias = pvias.merge(pvias_l, left_index=True, right_index=True).drop(['level_0', 'level_1'], axis=1)

    pvias_class = pvias.sjoin(class_viaria, op='contains', how='left')['cvc_classe'].reset_index().groupby('index').cvc_classe.agg('max')

    pvias = pvias.merge(pvias_class, left_index=True, right_index=True)

    pvias.to_file(f'resultado/pre-processamentos/{dist.ds_codigo}_poligono_de_vias_logradouros_{dist.ds_nome.lower().replace(" ", "_")}.gpkg', driver='GPKG')
        
    # break

MOEMA
PARQUE DO CARMO
PERDIZES
PIRITUBA
PONTE RASA
SANTO AMARO
SAO DOMINGOS
SAO MATEUS
SE
VILA LEOPOLDINA
JARDIM ANGELA
VILA SONIA
ANHANGUERA
ARTUR ALVIM
BRAS
CACHOEIRINHA
CAMBUCI
CAMPO LIMPO
CAPAO REDONDO
CASA VERDE
CIDADE DUTRA
CIDADE TIRADENTES
ERMELINO MATARAZZO
JARDIM SAO LUIS
SAUDE
CURSINO
VILA MARIANA
RAPOSO TAVARES
TUCURUVI
SAO LUCAS
SAPOPEMBA
CAMPO BELO
SAO MIGUEL
RIO PEQUENO
CARRAO
ALTO DE PINHEIROS
CIDADE LIDER
VILA FORMOSA
VILA GUILHERME
VILA JACUI
VILA MARIA
VILA MATILDE
VILA MEDEIROS
VILA PRUDENTE
PARELHEIROS
AGUA RASA
MARSILAC
MORUMBI
PEDREIRA
PENHA
PERUS
PINHEIROS
REPUBLICA
SACOMA
SANTANA
SAO RAFAEL
TATUAPE
VILA ANDRADE
VILA CURUCA
ARICANDUVA
GUAIANASES
IGUATEMI
ITAIM BIBI
ITAIM PAULISTA
JACANA
JAGUARA
JOSE BONIFACIO
LAJEADO
LAPA
LIBERDADE
LIMAO
IPIRANGA
CIDADE ADEMAR
TREMEMBE
JABAQUARA
ITAQUERA
MANDAQUI
MOOCA
PARI
SANTA CECILIA
SOCORRO
BARRA FUNDA
BELA VISTA
BELEM
BOM RETIRO
BRASILANDIA
BUTANTA
CAMPO GRANDE
CANGAIBA
CONSOLACAO
FREGUESIA DO O
GRAJAU
JAGUARE
JARAGUA
JARD

## Quantificando todos os distritos

In [122]:
pvias = [] 

for row, dist in df_dist.iterrows():
    p = gpd.read_file(f'resultado/pre-processamentos/{dist.ds_codigo}_poligono_de_vias_logradouros_{dist.ds_nome.lower().replace(" ", "_")}.gpkg')
    
    p['ds_codigo'] = dist.ds_codigo
    p['ds_nome'] = dist.ds_nome
    p['ds_cd_sub'] = dist.ds_cd_sub
    p['ds_subpref'] = dist.ds_subpref

    pvias.append(gpd.read_file(f'resultado/pre-processamentos/{dist.ds_codigo}_poligono_de_vias_logradouros_{dist.ds_nome.lower().replace(" ", "_")}.gpkg'))


In [123]:
len(pvias)

96

In [126]:
df_pvias = pd.concat(pvias)

In [129]:
gdf_pvias = gpd.GeoDataFrame(df_pvias, geometry=df_pvias.geometry, crs='EPSG:31983')

In [132]:
gdf_pvias.area.sum()

134817937.62937266

In [133]:
gdf_pvias.comprimento.sum()

12859202.534101734

In [134]:
gdf_pvias.area.sum() / gdf_pvias.comprimento.sum()

10.484160061391414